# CVRP optimal solver (Julia)
This notebook mirrors the Python formulation in `optimal_first_try.ipynb` but implements the parser and single-commodity flow model entirely in Julia so it can be run from an IJulia kernel.

In [1]:
using JuMP
using Gurobi
import MathOptInterface as MOI

In [2]:
struct VRPInstance
    name::String
    n::Int
    capacity::Int
    coords::Vector{Tuple{Float64,Float64}}
    demand::Vector{Int}
    depot::Int
    dist::Matrix{Float64}
end

function parse_vrplib(path::AbstractString)::VRPInstance
    name = ""; n = 0; cap = 0; edge_type = ""
    coords = Dict{Int,Tuple{Float64,Float64}}()
    demand = Dict{Int,Int}()
    depot = 1
    section = ""

    open(path, "r") do io
        for ln in eachline(io)
            s = strip(ln)
            isempty(s) && continue

            if occursin(":", s) && !(startswith(s, "NODE_COORD_SECTION")
               || startswith(s, "DEMAND_SECTION") || startswith(s, "DEPOT_SECTION"))
                key, val = strip.(split(s, ":", limit=2))
                ku = uppercase(key)
                if ku == "NAME"
                    name = val
                elseif ku == "DIMENSION"
                    n = parse(Int, split(val)[1])
                elseif ku == "CAPACITY"
                    cap = parse(Int, split(val)[1])
                elseif ku == "EDGE_WEIGHT_TYPE"
                    edge_type = strip(split(val)[1])
                end
                continue
            end

            up = uppercase(s)
            if up in ("NODE_COORD_SECTION","DEMAND_SECTION","DEPOT_SECTION","EOF")
                section = up
                continue
            end

            if section == "NODE_COORD_SECTION"
                p = split(s); idx = parse(Int, p[1])
                coords[idx] = (parse(Float64, p[2]), parse(Float64, p[3]))
            elseif section == "DEMAND_SECTION"
                p = split(s); idx = parse(Int, p[1])
                demand[idx] = parse(Int, p[2])
            elseif section == "DEPOT_SECTION"
                idx = parse(Int, split(s)[1])
                if idx != -1
                    depot = idx
                end
            end
        end
    end

    @assert n > 0 "DIMENSION not found in $path"
    @assert cap > 0 "CAPACITY not found in $path"

    coord_vec = [coords[i] for i in 1:n]
    dem_vec   = [get(demand, i, 0) for i in 1:n]

    # VRPLIB distance matrix
    dist = zeros(Float64, n, n)
    for i in 1:n, j in 1:n
        if i == j
            dist[i,j] = 0.0
        else
            dx = coord_vec[i][1] - coord_vec[j][1]
            dy = coord_vec[i][2] - coord_vec[j][2]
            d  = sqrt(dx*dx + dy*dy)
            et = uppercase(edge_type)
            if et == "EUC_2D"
                dist[i,j] = floor(d + 0.5)   # VRPLIB EUC_2D
            elseif et == "CEIL_2D"
                dist[i,j] = ceil(d)
            else
                dist[i,j] = d                # fallback
            end
        end
    end

    VRPInstance(name, n, cap, coord_vec, dem_vec, depot, dist)
end

parse_vrplib (generic function with 1 method)

In [3]:
function extract_routes_from_x(xmat::AbstractMatrix{<:Real}, depot::Int)
    n = size(xmat, 1)
    routes = Vector{Vector{Int}}()

    # starting arcs from depot
    starts = [j for j in 1:n if j != depot && xmat[depot, j] > 0.5]

    # successor map (strongest outgoing per node)
    succ = Dict{Int,Int}()
    for i in 1:n
        bestj, bestv = 0, 0.0
        for j in 1:n
            i == j && continue
            v = xmat[i, j]
            if v > bestv
                bestv = v; bestj = j
            end
        end
        if bestv > 0.5
            succ[i] = bestj
        end
    end

    for s in starts
        route = Int[depot, s]
        visited = Set([depot, s])
        cur = s
        while true
            nxt = get(succ, cur, depot)
            push!(route, nxt)
            if nxt == depot; break; end
            if nxt in visited
                push!(route, depot); break
            end
            push!(visited, nxt)
            cur = nxt
        end
        push!(routes, route)
    end
    return routes
end

function route_cost(route::Vector{Int}, inst::VRPInstance)
    c = 0.0
    for k in 1:length(route)-1
        c += inst.dist[route[k], route[k+1]]
    end
    c
end

function route_demand(route::Vector{Int}, inst::VRPInstance)
    tot = 0
    for k in 2:(length(route)-1)
        tot += inst.demand[route[k]]
    end
    tot
end

function route_total_cost(routes::Vector{Vector{Int}}, inst::VRPInstance)
    tot = 0.0
    for r in routes
        tot += route_cost(r, inst)
    end
    tot
end

function x_total_cost(xmat::AbstractMatrix{<:Real}, inst::VRPInstance)
    n = size(xmat, 1)
    c = 0.0
    for i in 1:n, j in 1:n
        if i != j && xmat[i, j] > 0.5
            c += inst.dist[i, j]
        end
    end
    c
end

function pretty_print_routes(routes::Vector{Vector{Int}}, inst::VRPInstance)
    if isempty(routes)
        println("No routes extracted (routes is empty).")
        return
    end
    println("Vehicles: ", length(routes))
    for (i, r) in enumerate(routes)
        println("Route #", i, ": ", join(r, " -> "),
                " | demand=", route_demand(r, inst),
                " | cost=", route_cost(r, inst))
    end
    println("Total cost = ", route_total_cost(routes, inst))
end

# Quick sanity checks
function check_capacity(routes::Vector{Vector{Int}}, inst::VRPInstance)
    viol = []
    for (i, r) in enumerate(routes)
        d = route_demand(r, inst)
        if d > inst.capacity
            push!(viol, (i, d))
        end
    end
    viol
end

function check_coverage(routes::Vector{Vector{Int}}, inst::VRPInstance)
    n = inst.n; depot = inst.depot
    custs = setdiff(collect(1:n), [depot])
    seen = Int[]
    for r in routes
        append!(seen, r[2:end-1])
    end
    missing = setdiff(custs, seen)
    dup = length(seen) - length(unique(seen))
    missing, dup
end

check_coverage (generic function with 1 method)

In [4]:
# Parse CVRPLIB .sol as customer sequences (no depot listed), plus a stated cost if present.
function parse_solution_customers_with_cost(path::AbstractString)
    routes_customers = Vector{Vector{Int}}()
    stated_cost::Union{Nothing,Float64} = nothing
    cost_pattern = r"(?i)\b(cost|distance|obj(?:ective)?)\b[^0-9\-]*(-?\d+(?:\.\d+)?)"

    open(path, "r") do io
        for ln in eachline(io)
            s = strip(ln)
            if startswith(lowercase(s), "route")
                parts = split(s, ":")
                nodes_str = length(parts) > 1 ? strip(parts[2]) : ""
                if !isempty(nodes_str)
                    custs = [parse(Int, t) for t in split(nodes_str) if !isempty(t)]
                    push!(routes_customers, custs)
                end
            else
                m = match(cost_pattern, s)
                if m !== nothing
                    stated_cost = parse(Float64, m.captures[2])
                end
            end
        end
    end
    return routes_customers, stated_cost
end

# Wrap customers with the actual depot from the .vrp file
build_routes_with_depot(routes_customers::Vector{Vector{Int}}, depot::Int) =
    [vcat([depot], filter(x -> x != depot, rc), [depot]) for rc in routes_customers]

build_routes_with_depot (generic function with 1 method)

In [5]:
mutable struct CVRPResult
    objective::Float64
    routes::Vector{Vector{Int}}
    xvals::Matrix{Float64}
    runtime_sec::Float64
    mip_gap::Float64
    vehicles::Int
    term_status::MOI.TerminationStatusCode
    primal_status::MOI.ResultStatusCode
end

function solve_cvrp_scf(inst::VRPInstance; K::Union{Nothing,Int}=nothing,
                        timelimit::Float64=180.0, mipgap::Float64=0.0,
                        seed::Int=0)

    n = inst.n
    depot = inst.depot
    V = 1:n
    customers = [i for i in V if i != depot]
    Q = inst.capacity

    model = Model(Gurobi.Optimizer)
    set_silent(model)
    set_optimizer_attribute(model, "TimeLimit", timelimit)
    set_optimizer_attribute(model, "MIPGap", mipgap)     # 0.0 => aim for proven optimum
    set_optimizer_attribute(model, "MIPFocus", 3)        # prioritize optimality
    set_optimizer_attribute(model, "Cuts", 2)
    set_optimizer_attribute(model, "Heuristics", 0.25)
    set_optimizer_attribute(model, "Presolve", 2)
    set_optimizer_attribute(model, "Seed", seed)

    @variable(model, x[i in V, j in V; i != j], Bin)
    @variable(model, f[i in V, j in V; i != j] >= 0)  # commodity flow (load)

    # Each customer in/out exactly once
    @constraint(model, [i in customers], sum(x[i,j] for j in V if j != i) == 1)
    @constraint(model, [j in customers], sum(x[i,j] for i in V if i != j) == 1)

    # Depot balance and optional vehicle cap
    @constraint(model, sum(x[depot,j] for j in V if j != depot) == sum(x[i,depot] for i in V if i != depot))
    if K !== nothing
        @constraint(model, sum(x[depot,j] for j in V if j != depot) <= K)
    end

    # Single-commodity flow: sum_in(v) - sum_out(v) = b[v]
    b = zeros(Float64, n)
    b[depot] = -sum(inst.demand[i] for i in customers)
    for i in customers
        b[i] = inst.demand[i]
    end
    @constraint(model, [v in V],
        sum(f[i,v] for i in V if i != v) - sum(f[v,j] for j in V if j != v) == b[v])

    # Link flow to capacity & arc usage
    @constraint(model, [i in V, j in V; i != j], f[i,j] <= Q * x[i,j])

    # (Optional strengthening) Forbid 2-customer cycles
    @constraint(model, [i in customers, j in customers; i < j], x[i,j] + x[j,i] <= 1)

    @objective(model, Min, sum(inst.dist[i,j] * x[i,j] for i in V for j in V if i != j))

    runtime = @elapsed optimize!(model)

    term = termination_status(model)
    prim = primal_status(model)
    rc = MOI.get(model, MOI.ResultCount())
    obj = rc > 0 ? objective_value(model) : NaN

    xmat = zeros(Float64, n, n)
    if rc > 0
        for i in V, j in V
            if i != j
                xv = value(x[i,j])
                xmat[i,j] = isnan(xv) ? 0.0 : xv
            end
        end
    end

    routes = rc > 0 ? extract_routes_from_x(xmat, depot) : Vector{Vector{Int}}()
    vehs = rc > 0 ? length(routes) : 0
    gap = rc > 0 ? try MOI.get(model, MOI.RelativeGap()) catch; NaN end : NaN

    CVRPResult(obj, routes, xmat, runtime, gap, vehs, term, prim)
end

solve_cvrp_scf (generic function with 1 method)

In [6]:
# === Batch solve (SCF) + print details + export CSV (one cell) ===
# Assumes you've already run the cells that define:
#   - parse_vrplib(path)::VRPInstance
#   - solve_cvrp_scf(inst; K, timelimit, mipgap, seed)::CVRPResult

@assert isdefined(Main, :parse_vrplib) "Missing parse_vrplib. Run your VRPLIB parser cell first."
@assert isdefined(Main, :solve_cvrp_scf) "Missing solve_cvrp_scf. Run your SCF solver cell first."

# Choose the folder to scan (use existing `folder` if you've set it; else current dir)
folder = @isdefined(folder) ? folder : "A"
@assert isdir(folder) "Not a directory: $folder"

# --- Helpers (local to this cell) ---
function _basename(p::AbstractString)
    s = replace(p, "\\" => "/")
    i = findlast('/', s)                # return Int index
    return i === nothing ? s : s[(i+1):end]
end

_parseK(path::AbstractString) = begin
    m = match(r"-k(\d+)\.vrp$"i, path)
    m === nothing ? nothing : parse(Int, m.captures[1])
end

function _stated_cost(sol_path::AbstractString)
    cp = r"(?i)\b(cost|distance|obj(?:ective)?)\b[^0-9\-]*(-?\d+(?:\.\d+)?)"
    sc::Union{Nothing,Float64} = nothing
    open(sol_path, "r") do io
        for ln in eachline(io)
            m = match(cp, strip(ln))
            if m !== nothing
                sc = parse(Float64, m.captures[2])
            end
        end
    end
    return sc
end

# Collect .vrp files
vrp_files = sort(filter(f -> endswith(lowercase(f), ".vrp"),
                       readdir(folder; join=true)))

if isempty(vrp_files)
    println("No .vrp files found in: $folder")
else
    # Accumulator for CSV rows
    results = NamedTuple[]

    println("Batch folder: $folder")
    for (k, vrp_path) in enumerate(vrp_files)
        name = _basename(vrp_path)
        println("\n[$k/$(length(vrp_files))] $name")

        # Load instance and set limits
        inst = parse_vrplib(vrp_path)
        K    = _parseK(vrp_path)
        tl   = 20.0 * inst.n   # TimeLimit = 10 * n (seconds)

        # Solve (SCF)
        res = solve_cvrp_scf(inst; K=K, timelimit=tl, mipgap=0.0, seed=0)

        # Print solver info
        println("  n=$(inst.n), K=$(K), TL=$(Int(round(tl)))s, capacity=$(inst.capacity)")
        println("  solver objective = ", res.objective,
                " | vehicles = ", res.vehicles,
                " | gap = ", res.mip_gap,
                " | runtime(s) = ", round(res.runtime_sec, digits=2))
        println("  status: term=", res.term_status, " | primal=", res.primal_status)

        # Compare to stated .sol cost (no recomputation)
        sol_path = replace(vrp_path, ".vrp" => ".sol")
        sc = isfile(sol_path) ? _stated_cost(sol_path) : nothing
        if sc !== nothing && !isnan(res.objective)
            diff  = res.objective - sc
            rdiff = 100 * diff / sc
            println("  stated cost (.sol) = ", sc,
                    " | diff = ", diff,
                    " | diff% = ", round(rdiff, digits=3))
        elseif isfile(sol_path)
            println("  stated cost present but solver has no incumbent under limits.")
        else
            println("  paired .sol not found; skipping stated-cost compare.")
        end

        # Save row for CSV
        push!(results, (
            name = name,
            n = inst.n,
            K = K,
            capacity = inst.capacity,
            timelimit = tl,
            objective = res.objective,
            mip_gap = res.mip_gap,
            runtime = res.runtime_sec,
            vehicles = res.vehicles,
            term = string(res.term_status),
            primal = string(res.primal_status),
            stated_cost = sc,
            diff = (sc === nothing || isnan(res.objective)) ? nothing : (res.objective - sc),
            diff_pct = (sc === nothing || isnan(res.objective)) ? nothing : (100 * (res.objective - sc) / sc),
        ))
    end

    # Write CSV next to the data
    outcsv = joinpath(folder, "batch_results20n.csv")
    open(outcsv, "w") do io
        println(io, "name,n,K,capacity,timelimit,objective,mip_gap,runtime,vehicles,term,primal,stated_cost,diff,diff_pct")
        for r in results
            println(io, string(
                r.name, ",", r.n, ",", (r.K === nothing ? "" : r.K), ",", r.capacity, ",",
                r.timelimit, ",", r.objective, ",", r.mip_gap, ",", r.runtime, ",",
                r.vehicles, ",", r.term, ",", r.primal, ",",
                (r.stated_cost === nothing ? "" : r.stated_cost), ",",
                (r.diff === nothing ? "" : r.diff), ",",
                (r.diff_pct === nothing ? "" : r.diff_pct)
            ))
        end
    end
    println("\nWrote summary CSV: ", outcsv)
end

Batch folder: A

[1/27] A-n32-k5.vrp
Set parameter Username
Set parameter LicenseID to value 2696413
Academic license - for non-commercial use only - expires 2026-08-18
  n=32, K=5, TL=640s, capacity=100
  solver objective = 784.0 | vehicles = 5 | gap = 0.0 | runtime(s) = 36.59
  status: term=OPTIMAL | primal=FEASIBLE_POINT
  stated cost (.sol) = 784.0 | diff = 0.0 | diff% = 0.0

[2/27] A-n33-k5.vrp
Set parameter Username
Set parameter LicenseID to value 2696413
Academic license - for non-commercial use only - expires 2026-08-18
  n=33, K=5, TL=660s, capacity=100
  solver objective = 661.0 | vehicles = 5 | gap = 0.0 | runtime(s) = 31.12
  status: term=OPTIMAL | primal=FEASIBLE_POINT
  stated cost (.sol) = 661.0 | diff = 0.0 | diff% = 0.0

[3/27] A-n33-k6.vrp
Set parameter Username
Set parameter LicenseID to value 2696413
Academic license - for non-commercial use only - expires 2026-08-18
  n=33, K=6, TL=660s, capacity=100
  solver objective = 742.0 | vehicles = 6 | gap = 0.0 | runtime(